#### Draw R2 change

In [2]:
# Imports
import os
import numpy as np
import nibabel as nb
import pandas as pd
import warnings
import pingouin

warnings.filterwarnings('ignore')

# Figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)

# settings
subjects = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
            'sub-005', 'sub-006', 'sub-007', 'sub-008']
subjects_plot = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
                 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'group']
tasks = ['FullScreen', 'FullScreenAttendFix', 'FullScreenAttendBar']
rois = ['V1', 'V2', 'V3', 'V3AB', 'hMT+', 'LO',
        'VO', 'iIPS', 'sIPS', 'iPCS', 'sPCS', 'mPCS']

best_voxels_num = 250

# general figure settings
template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",
                        axes_width=2,
                        axes_font_size=13,
                        bg_col="rgba(255, 255, 255, 1)",
                        font='Arial',
                        title_font_size=15,
                        plot_width=1.5)

In [13]:
# load TSV
for subject in subjects:
    tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject)
    df_fn = "{}/{}_all_res_best{}.pkl".format(tsv_dir,subject,int(best_voxels_num))
    df_sub = pd.read_pickle(df_fn)
    
    # compute mean change relative to gaze center
    df_new = pd.DataFrame({'subject': df_sub.subject}) 
    df_new['roi'] = df_sub.roi
    df_new['r2_change_ab']= ((df_sub.retino_pred_gl_ab_r2 - df_sub.retino_pred_gc_ab_r2) + (df_sub.retino_pred_gr_ab_r2 - df_sub.retino_pred_gc_ab_r2)/2.0)
    df_new['r2_change_af']= ((df_sub.retino_pred_gl_af_r2 - df_sub.retino_pred_gc_af_r2) + (df_sub.retino_pred_gr_af_r2 - df_sub.retino_pred_gc_af_r2)/2.0)
    
    # across roi
    if roi_num > 0: df2 = pd.concat([df2,df_roi], ignore_index=True)
    else: df2 = df_roi
    
    

In [ ]:
# Subplot settings
rows, cols = 1, 2
fig_height, fig_width = 800,400
column_widths,row_heights = [1,1],[1]
sb_specs = [[{}],[{}]]
hovertemplate_task = 'ROI: %{y}<br>' + 'R2: %{x:1.2f}'



xaxis_range = [0,0.8]
xaxis_dtick = 0.2

x_gap_line = 0.1*xaxis_range[1]
x_gap_asterisk = 0.15*xaxis_range[1]
    
for subject in subjects_plot:
    subplot_titles = ['<b>Attention effect</b> ({})'.format(subject)]

    fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.15, horizontal_spacing=0.05, 
                    column_widths=column_widths, row_heights=row_heights,  subplot_titles=subplot_titles)

    # Get Data
    tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject)
    af_df = pd.read_csv("{}/{}_task-FullScreenAttendFix_prf_threshold_par{}.tsv".format(tsv_dir,subject,type_analysis), sep="\t")
    ab_df = pd.read_csv("{}/{}_task-FullScreenAttendBar_prf_threshold_par{}.tsv".format(tsv_dir,subject,type_analysis), sep="\t")
    af_df.roi = pd.Categorical(af_df.roi,categories=rois)
    ab_df.roi = pd.Categorical(af_df.roi,categories=rois)

    x_af, x_ab = af_df.groupby('roi').r2.mean(), ab_df.groupby('roi').r2.mean()
    if subject == 'group': 
        x_af_eb = af_df.groupby(['roi','subject']).r2.mean().reset_index(name='r2').groupby('roi').r2.std()/np.sqrt(len(subjects)-1)
        x_ab_eb = ab_df.groupby(['roi','subject']).r2.mean().reset_index(name='r2').groupby('roi').r2.std()/np.sqrt(len(subjects)-1)
    else: 
        x_af_eb, x_ab_eb = af_df.groupby('roi').r2.std(), ab_df.groupby('roi').r2.std()


    # bar plot
    fig.append_trace(go.Bar(y=rois, x=x_ab, orientation='h',
                            showlegend=True, legendgroup='attend-bar', name='<i>attend-bar<i>',
                            marker_color=ab_line_col, hovertemplate=hovertemplate_task,
                            error_x_type='data', error_x_symmetric=False, error_x_array=x_ab_eb, error_x_arrayminus=x_ab_eb),row=1, col=1)

    fig.append_trace(go.Bar(y=rois, x=x_af, orientation='h',
                            showlegend=True, legendgroup='attend-fix', name='<i>attend-fix<i>', 
                            marker_color=af_line_col, hovertemplate=hovertemplate_task,
                            error_x_type='data', error_x_symmetric=False, error_x_array=x_af_eb, error_x_arrayminus=x_af_eb),row=1, col=1)


    # get Stats
    if subject == 'group': 
        stats_df = pd.read_csv("{}/group_task-FullScreen_prf_threshold_stats{}.tsv".format(tsv_dir,type_analysis), sep="\t")
        p_flag = stats_df.p_val_unilateral<0.05
        fig.append_trace(go.Scatter(x=np.max((x_ab,x_af),axis=0)[p_flag]+x_gap_line, y=np.array(rois)[p_flag],
                                    marker_symbol='line-ns-open',
                                    showlegend=False, mode='markers', marker_color=ab_line_col, hoverinfo='skip',
                                    line_width=1, opacity=1, marker_size= 15),row=1, col=1)
        fig.append_trace(go.Scatter(x=np.max((x_ab,x_af),axis=0)[p_flag]+x_gap_asterisk, y=np.array(rois)[p_flag],
                                    marker_symbol='asterisk-open',
                                    showlegend=False, mode='markers', marker_color=ab_line_col, hoverinfo='skip',
                                    line_width=1, opacity=1, marker_size= 10),row=1, col=1)

    # set axis
    for row in np.arange(rows):
        for col in np.arange(cols):
            fig.update_xaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)
            fig.update_yaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1, autorange='reversed')

    # set figure
    fig.layout.update(xaxis_range=xaxis_range, xaxis_title='R2', xaxis_dtick=xaxis_dtick,
                      yaxis_title='',bargap=0.30,
                      template=fig_template, width=fig_width, height=fig_height, margin_l=50, margin_r=20, margin_t=50, margin_b=70,
                      legend_yanchor='top', 
                      legend_y=0.995, legend_xanchor='left', legend_x=0.9, legend_bgcolor='rgba(255,255,255,0)', legend_tracegroupgap=1)

    # show and save figure
    fig.show(config={"displayModeBar": False})
    # fig.write_image("{}/{}_attcmp{}.pdf".format(tsv_dir, subject, type_analysis))
    # fig.write_html("{}/{}_attcmp{}.html".format(tsv_dir, subject, type_analysis),config={"displayModeBar": False})